# Build ResNet50 with Keras
The original [paper](https://arxiv.org/abs/1512.03385). ResNet Configuration:
![ResNet](./JupyterImageSupport/ResNet.jpg)

In [1]:
from keras.layers import Conv2D, MaxPool2D,BatchNormalization, Input, GlobalAveragePooling2D
from keras.layers import ReLU, add, ZeroPadding2D, Dense
from keras.models import Model 

Using TensorFlow backend.


In [2]:
def BN_ReLU_layer(x):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x


def residual_block(x, nb_input_channel, nb_output_channel, strides=(1, 1), need_shortcut=False):

        shortcut = x
        if need_shortcut or strides != (1, 1):
            shortcut = Conv2D(nb_output_channel, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
            shortcut = BatchNormalization()(shortcut)

    
        x = Conv2D(nb_input_channel, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
        x = BN_ReLU_layer(x)

        x = Conv2D(nb_input_channel, kernel_size=(3, 3), strides=strides, padding='same')(x)
        x = BN_ReLU_layer(x)

        x = Conv2D(nb_output_channel, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        

        x = add([shortcut,x])
        x = ReLU()(x)

        return x

In [3]:
img_height,img_width,img_channels = 224,224,3
img_image_shape = (img_height,img_width,img_channels)
img_input = Input(shape=img_image_shape )

# Conv 1
#x = ZeroPadding2D()(img_input)
x = Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same')(img_input)
x = BN_ReLU_layer(x)

# Conv 2
x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
for i in range(3):
    need_shortcut = True if i == 0 else False
    x = residual_block(x,64,256,need_shortcut=need_shortcut)
    
# At each begining of the following Conv block,we apply down_sampling with strides=(2,2)
# Conv 3
for i in range(4):
    strides = (2, 2) if i == 0 else (1, 1)
    x = residual_block(x,128,512,need_shortcut=True,strides=strides)

# Conv 4
for i in range(6):
    strides = (2, 2) if i == 0 else (1, 1)
    x = residual_block(x,256,1024,need_shortcut=True,strides=strides)

# Conv 4
for i in range(3):
    strides = (2, 2) if i == 0 else (1, 1)
    x = residual_block(x,512,2048,need_shortcut=True,strides=strides)


x = GlobalAveragePooling2D()(x)
x = Dense(1000,activation='softmax')(x)

model = Model(inputs=img_input,outputs=x)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin